In [ ]:
import pandas as pd
import os
import numpy as np
import pickle
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from collections import defaultdict
from keras.models import Sequential
from keras.layers import Input, Dense, LSTM, SimpleRNN, Dropout
from keras.layers import GRU, Activation, Reshape, LeakyReLU
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
matplotlib.font_manager._rebuild()

plt.rcParams["font.family"] = 'NanumGothic'

In [ ]:
trainf = pd.read_csv('/home/jovyan/WIDE_HOME/Competition/data/경진대회데이터/더존비즈온(매출예측라벨포함)/재무통계(월별)_train.csv',dtype={'cd_biztyp4':str})
cols = ['year', '분기', 'yn_private', '자산_현금', '자산_당좌예금', '자산_보통예금', '자산_기타제예금', '자산_정기예.적금', '자산_기타단기금융상품', '자산_단기매매증권', '자산_외상매출금', '자산_받을어음', '자산_공사미수금', '자산_단기대여금', '자산_미수수익', '자산_분양미수금', '자산_미수금', '자산_소모품', '자산_매도가능증권', '자산_만기보유증권', '자산_선급금', '자산_선급비용', '자산_가지급금', '자산_부가세대급금', '자산_선납세금', '자산_주.임.종단기채권', '자산_전도금', '자산_선급공사비', '자산_이연법인세자산', '자산_상품', '자산_제품', '자산_완성건물', '자산_원재료', '자산_원재료(도급)', '자산_원재료(분양)', '자산_부재료', '자산_건설용지', '자산_가설재', '자산_저장품', '자산_미착품', '자산_재공품', '자산_미완성공사(도급)', '자산_미완성공사(분양)', '자산_장기성예금', '자산_특정현금과예금', '자산_매도가능증권.1', '자산_장기대여금', '자산_만기보유증권.1', '자산_지분법적용투자주식', '자산_투자부동산', '자산_단체퇴직보험예치금', '자산_투자일임계약자산', '자산_퇴직연금운용자산', '자산_토지', '자산_건물', '자산_구축물', '자산_기계장치', '자산_차량운반구', '자산_공구와기구', '자산_비품', '자산_건설중인자산', '자산_미착기계', '자산_건설용장비', '자산_영업권', '자산_특허권', '자산_상표권', '자산_실용신안권', '자산_의장권', '자산_면허권', '자산_광업권', '자산_창업비', '자산_개발비', '자산_소프트웨어', '자산_이연법인세자산.1', '자산_임차보증금', '자산_전세권', '자산_기타보증금', '자산_장기외상매출금', '자산_장기받을어음', '자산_장기미수금', '자산_장기선급비용', '자산_장기선급금', '자산_부도어음과수표', '자산_전신전화가입권', '부채_외상매입금', '부채_지급어음', '부채_미지급금', '부채_예수금', '부채_부가세예수금', '부채_당좌차월', '부채_가수금', '부채_예수보증금', '부채_선수금', '부채_단기차입금', '부채_미지급세금', '부채_미지급비용', '부채_선수수익', '부채_유동성장기부채', '부채_미지급배당금', '부채_지급보증채무', '부채_수출금융', '부채_수입금융', '부채_공사손실충당금', '부채_하자보수충당금', '부채_공사선수금', '부채_분양선수금', '부채_이연법인세부채', '부채_사채', '부채_장기차입금', '부채_임대보증금', '부채_퇴직급여충당부채', '부채_퇴직보험충당부채', '부채_장기미지급금', '부채_중소기업투자준비금', '부채_기술개발준비금', '부채_해외시장개척준비금', '부채_지방이전준비금', '부채_수출손실준비금', '부채_주.임.종장기차입금', '부채_관계회사장기차입금', '부채_외화장기차입금', '부채_장기공사선수금', '부채_장기임대보증금', '부채_장기성지급어음', '부채_전환사채', '부채_퇴직연금충당부채', '자본_자본금', '자본_우선주자본금', '자본_출자금', '자본_인출금', '자본_주식발행초과금', '자본_기타자본잉여금', '자본_이익준비금', '자본_기업합리화적립금', '자본_재무구조개선적립금', '자본_임의적립금', '자본_사업확장적립금', '자본_감채적립금', '자본_배당평균적립금', '자본_자기주식처분손잔액', '자본_중소기업투자준비금', '자본_기술개발준비금', '자본_전기오류수정이익', '자본_전기오류수정손실', '자본_중간배당금', '자본_이월이익잉여금', '자본_이월결손금', '자본_미처분이익잉여금', '자본_미처리결손금', '자본_주식할인발행차금', '자본_자기주식', '자본_전환권대가', '자본_국고보조금', '자본_손익', '자본_매도가능증권평가익', '자본_매도가능증권평가손', '자본_파생상품평가손실', '매출_상품매출', '매출_제품매출', '매출_공사수입금', '매출_완성건물매출', '매출_임대료수입', '제조_급여', '제조_임금', '제조_상여금', '제조_제수당', '제조_잡급', '제조_퇴직급여', '제조_퇴직보험충당금전입', '제조_퇴직급여.1', '제조_복리후생비', '제조_여비교통비', '제조_접대비', '제조_통신비', '제조_가스수도료', '제조_전력비', '제조_세금과공과금', '제조_감가상각비', '제조_지급임차료', '제조_수선비', '제조_보험료', '제조_차량유지비', '제조_경상연구개발비', '제조_운반비', '제조_교육훈련비', '제조_도서인쇄비', '제조_회의비', '제조_포장비', '제조_사무용품비', '제조_소모품비', '제조_지급수수료', '제조_보관료', '제조_외주가공비', '제조_잡비', '도급_외주비', '도급_급여', '도급_임금', '도급_상여금', '도급_제수당', '도급_퇴직급여', '도급_퇴직급여.1', '도급_중기및운반비', '도급_복리후생비', '도급_여비교통비', '도급_접대비', '도급_통신비', '도급_가스수도료', '도급_전력비', '도급_세금과공과금', '도급_감가상각비', '도급_지급임차료', '도급_수선비', '도급_보험료', '도급_차량유지비', '도급_운반비', '도급_잡자재대', '도급_교육훈련비', '도급_도서인쇄비', '도급_사무용품비', '도급_소모품비', '도급_지급수수료', '도급_외주가공비', '도급_장비사용료', '도급_설계용역비', '도급_광고선전비', '도급_소모공구비', '도급_외주공사비', '도급_협회비', '도급_잡비', '분양_급여', '분양_임금', '분양_복리후생비', '분양_가스수도료', '분양_전력비', '분양_세금과공과금', '분양_보험료', '분양_차량유지비', '분양_소모품비', '분양_지급수수료', '분양_설계용역비', '분양_외주비', '판관비_임원급여', '판관비_직원급여', '판관비_상여금', '판관비_제수당', '판관비_잡급', '판관비_퇴직급여', '판관비_퇴직보험충당금전입', '판관비_퇴직급여.1', '판관비_복리후생비', '판관비_여비교통비', '판관비_접대비', '판관비_통신비', '판관비_수도광열비', '판관비_전력비', '판관비_세금과공과금', '판관비_감가상각비', '판관비_지급임차료', '판관비_수선비', '판관비_보험료', '판관비_차량유지비', '판관비_경상연구개발비', '판관비_운반비', '판관비_교육훈련비', '판관비_도서인쇄비', '판관비_회의비', '판관비_포장비', '판관비_사무용품비', '판관비_소모품비', '판관비_지급수수료', '판관비_보관료', '판관비_광고선전비', '판관비_판매촉진비', '판관비_대손상각비', '판관비_기밀비', '판관비_건물관리비', '판관비_수출제비용', '판관비_판매수수료', '판관비_무형고정자산상각', '판관비_환가료', '판관비_견본비', '판관비_해외접대비', '판관비_해외시장개척비', '판관비_미분양주택관리비', '판관비_잡비', '판관비_명예퇴직금', '기타_이자수익', '기타_배당금수익', '기타_수입임대료', '기타_단기투자자산평가익', '기타_외환차익', '기타_대손충당금환입', '기타_외화환산이익', '기타_사채상환이익', '기타_전기오류수정이익', '기타_유형자산처분이익', '기타_투자자산처분이익', '기타_상각채권추심이익', '기타_채무면제이익', '기타_보험차익', '기타_투자증권손상차환입', '기타_잡이익', '기타_이자비용', '기타_외환차손', '기타_기부금', '기타_기타의대손상각비', '기타_외화환산손실', '기타_단기투자자산평가손', '기타_단기투자자산처분손', '기타_재고자산평가손실', '기타_재해손실', '기타_전기오류수정손실', '기타_투자증권손상차손', '기타_지분법손실', '기타_보상비', '기타_유형자산처분손실', '기타_투자자산처분손실', '기타_잡손실', '기타_중단손익', '기타_법인세등', '기타_소득세등', '자산_현금_cumsum', '자산_당좌예금_cumsum', '자산_보통예금_cumsum', '자산_기타제예금_cumsum', '자산_정기예.적금_cumsum', '자산_기타단기금융상품_cumsum', '자산_단기매매증권_cumsum', '자산_외상매출금_cumsum', '자산_받을어음_cumsum', '자산_공사미수금_cumsum', '자산_단기대여금_cumsum', '자산_미수수익_cumsum', '자산_분양미수금_cumsum', '자산_미수금_cumsum', '자산_소모품_cumsum', '자산_매도가능증권_cumsum', '자산_만기보유증권_cumsum', '자산_선급금_cumsum', '자산_선급비용_cumsum', '자산_가지급금_cumsum', '자산_부가세대급금_cumsum', '자산_선납세금_cumsum', '자산_주.임.종단기채권_cumsum', '자산_전도금_cumsum', '자산_선급공사비_cumsum', '자산_이연법인세자산_cumsum', '자산_상품_cumsum', '자산_제품_cumsum', '자산_완성건물_cumsum', '자산_원재료_cumsum', '자산_원재료(도급)_cumsum', '자산_원재료(분양)_cumsum', '자산_부재료_cumsum', '자산_건설용지_cumsum', '자산_가설재_cumsum', '자산_저장품_cumsum', '자산_미착품_cumsum', '자산_재공품_cumsum', '자산_미완성공사(도급)_cumsum', '자산_미완성공사(분양)_cumsum', '자산_장기성예금_cumsum', '자산_특정현금과예금_cumsum', '자산_매도가능증권_cumsum.1', '자산_장기대여금_cumsum', '자산_만기보유증권_cumsum.1', '자산_지분법적용투자주식_cumsum', '자산_투자부동산_cumsum', '자산_단체퇴직보험예치금_cumsum', '자산_투자일임계약자산_cumsum', '자산_퇴직연금운용자산_cumsum', '자산_토지_cumsum', '자산_건물_cumsum', '자산_구축물_cumsum', '자산_기계장치_cumsum', '자산_차량운반구_cumsum', '자산_공구와기구_cumsum', '자산_비품_cumsum', '자산_건설중인자산_cumsum', '자산_미착기계_cumsum', '자산_건설용장비_cumsum', '자산_영업권_cumsum', '자산_특허권_cumsum', '자산_상표권_cumsum', '자산_실용신안권_cumsum', '자산_의장권_cumsum', '자산_면허권_cumsum', '자산_광업권_cumsum', '자산_창업비_cumsum', '자산_개발비_cumsum', '자산_소프트웨어_cumsum', '자산_이연법인세자산_cumsum.1', '자산_임차보증금_cumsum', '자산_전세권_cumsum', '자산_기타보증금_cumsum', '자산_장기외상매출금_cumsum', '자산_장기받을어음_cumsum', '자산_장기미수금_cumsum', '자산_장기선급비용_cumsum', '자산_장기선급금_cumsum', '자산_부도어음과수표_cumsum', '자산_전신전화가입권_cumsum', '부채_외상매입금_cumsum', '부채_지급어음_cumsum', '부채_미지급금_cumsum', '부채_예수금_cumsum', '부채_부가세예수금_cumsum', '부채_당좌차월_cumsum', '부채_가수금_cumsum', '부채_예수보증금_cumsum', '부채_선수금_cumsum', '부채_단기차입금_cumsum', '부채_미지급세금_cumsum', '부채_미지급비용_cumsum', '부채_선수수익_cumsum', '부채_유동성장기부채_cumsum', '부채_미지급배당금_cumsum', '부채_지급보증채무_cumsum', '부채_수출금융_cumsum', '부채_수입금융_cumsum', '부채_하자보수충당금_cumsum', '부채_공사선수금_cumsum', '부채_분양선수금_cumsum', '부채_이연법인세부채_cumsum', '부채_사채_cumsum', '부채_장기차입금_cumsum', '부채_임대보증금_cumsum', '부채_퇴직급여충당부채_cumsum', '부채_퇴직보험충당부채_cumsum', '부채_장기미지급금_cumsum', '부채_중소기업투자준비금_cumsum', '부채_기술개발준비금_cumsum', '부채_해외시장개척준비금_cumsum', '부채_주.임.종장기차입금_cumsum', '부채_관계회사장기차입금_cumsum', '부채_외화장기차입금_cumsum', '부채_장기공사선수금_cumsum', '부채_장기임대보증금_cumsum', '부채_장기성지급어음_cumsum', '부채_이연법인세부채_cumsum.1', '부채_신주인수권부사채_cumsum', '부채_전환사채_cumsum', '부채_장기제품보증부채_cumsum', '부채_퇴직연금충당부채_cumsum', '부채_퇴직연금미지급금_cumsum', '자본_자본금_cumsum', '자본_우선주자본금_cumsum', '자본_출자금_cumsum', '자본_인출금_cumsum', '자본_주식발행초과금_cumsum', '자본_감자차익_cumsum', '자본_자기주식처분이익_cumsum', '자본_기타자본잉여금_cumsum', '자본_재평가적립금_cumsum', '자본_이익준비금_cumsum', '자본_기업합리화적립금_cumsum', '자본_법정적립금임의설정_cumsum', '자본_재무구조개선적립금_cumsum', '자본_임의적립금_cumsum', '자본_사업확장적립금_cumsum', '자본_감채적립금_cumsum', '자본_배당평균적립금_cumsum', '자본_주식할인발행차상각_cumsum', '자본_상환주식의상환액_cumsum', '자본_자기주식처분손잔액_cumsum', '자본_중소기업투자준비금_cumsum', '자본_기술개발준비금_cumsum', '자본_해외시장개척준비금_cumsum', '자본_지방이전준비금_cumsum', '자본_수출손실준비금_cumsum', '자본_기타임의적립금_cumsum', '자본_회계변경의누적효과_cumsum', '자본_전기오류수정이익_cumsum', '자본_전기오류수정손실_cumsum', '자본_중간배당금_cumsum', '자본_기타이익잉여금_cumsum', '자본_이월이익잉여금_cumsum', '자본_이월결손금_cumsum', '자본_미처분이익잉여금_cumsum', '자본_미처리결손금_cumsum', '자본_당기순이익_cumsum', '자본_당기순손실_cumsum', '자본_주식할인발행차금_cumsum', '자본_자기주식_cumsum', '자본_전환권대가_cumsum', '자본_신주발행비_cumsum', '자본_신주청약증거금_cumsum', '자본_국고보조금_cumsum', '자본_공사부담금_cumsum', '자본_감자차손_cumsum', '자본_자기주식처분손실_cumsum', '자본_손익_cumsum', '자본_매도가능증권평가익_cumsum', '자본_매도가능증권평가손_cumsum', '자본_해외사업환산손실_cumsum', '매출거래처수', '매입거래처수', '부채_환율조정대', '부채_이연법인세부채.1', '부채_신주인수권부사채', '부채_장기제품보증부채', '부채_퇴직연금미지급금', '자본_감자차익', '자본_자기주식처분이익', '자본_재평가적립금', '자본_주식할인발행차상각', '자본_상환주식의상환액', '자본_지방이전준비금', '자본_수출손실준비금', '자본_신주인수권대가', '자본_신주청약증거금', '자본_해외사업환산이익', '매출원가_제조', '제조_시험비', '제조_기밀비', '도급_보관료', '분양_상여금', '분양_퇴직급여', '분양_여비교통비', '분양_접대비', '분양_통신비', '분양_지급임차료', '판관비_수주비', '기타_만기보유증권이자', '기타_단기투자자산처분익', '기타_수입수수료', '기타_자산수증이익', '기타_수출손실준비금환입', '기타_재고자산감모손실', '부채_공사손실충당금_cumsum', '자본_배당건설이자상각_cumsum', '자본_주식매입선택권_cumsum', '자본_해외사업환산이익_cumsum', '자본_파생상품평가이익_cumsum', '자본_파생상품평가손실_cumsum', '자본_법정적립금임의설정', '자본_배당건설이자상각', '자본_해외시장개척준비금', '자본_기타임의적립금', '자본_회계변경의누적효과', '자본_기타이익잉여금', '자본_당기순이익', '자본_당기순손실', '자본_배당건설이자', '자본_신주발행비', '자본_자기주식처분손실', '자본_주식매입선택권', '자본_해외사업환산손실', '도급_포장비', '판관비_하자보수충당금전입', '기타_기술개발준비금환입', '기타_해외개척준비금환입', '기타_지방이전준비금환입', '기타_중소투자준비금전입', '기타_기술개발준비금전입', '기타_해외개척준비금전입', '기타_지방이전준비금전입', '기타_수출손실준비금전입', '기타_특별상각', '가지급금건수', '차입건수', '매출_거래건수', '매입_거래건수', '매출금액', '매입금액','label']
train1 = trainf[cols]


In [ ]:
# 상관관계 히트맵 
# for i in range(20):
#     a = train1.iloc[:,(i*30):((i+1)*30)]
#     b = pd.concat([a,train1['label']])
#     plt.figure(figsize=(30, 30))
#     sns.heatmap(b.corr(),annot=True) 

In [ ]:
# 상관관계 지수가 제일 높은것만 우선적 선발  
data = ['자산_제품','자본_기술개발준비금','자산_외상매출금_cumsum','자산_선급금_cumsum','자산_부가세대급금_cumsum','자산_상품_cumsum','부채_외상매입금_cumsum','부채_부가세예수금_cumsum','자본_이월이익잉여금_cumsum','매입_거래건수','매출금액','매입금액','label']
t1 = trainf[data]
t1 = t1.iloc[:,3:]

In [ ]:
plt.figure(figsize=(15,15))
a = sns.heatmap(t1.corr(),annot=True)
plt.title('상관관계 히트맵', fontsize=30)
plt.show() 

In [ ]:
# 상관관계가 제일 높은 컬럼 선발 
cols1 = ['매출금액','매입금액','label']
cols2 = ['매출금액','매입금액']

def save_pickle(save_path, data):
    with open(save_path, 'wb') as w:
        pickle.dump(data, w)

train = trainf[cols1]

In [ ]:
# 딥러닝 알고리즘 모델 생성, 데이터를 3차원으로 변환후 학습 
train_df, valid_df = train_test_split(train, test_size=0.2) 

train_X, train_Y = np.array(train_df.iloc[:,:-1]), np.array(train_df['label']) 
valid_X, valid_Y = np.array(valid_df.iloc[:,:-1]), np.array(valid_df['label'])

scaler = MinMaxScaler()

scaler.fit(train_X)
train_X_scaled = scaler.transform(train_X)
valid_X_scaled = scaler.transform(valid_X)

train_X = train_X_scaled
valid_X = valid_X_scaled

train_X = train_X.reshape((train_X.shape[0],train_X.shape[1],1))
valid_X = valid_X.reshape((valid_X.shape[0],valid_X.shape[1],1))


In [ ]:
batch_size = 128
epochs = 50
output_size=1

model = Sequential()
model.add(GRU(24, input_shape=(2,1), kernel_initializer='uniform'))
model.add(Activation('relu'))
model.add(Dense(8))
model.add(LeakyReLU(0.2))
model.add(Dropout(0.3))
model.add(Dense(4))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(output_size))
model.compile(loss='mse', optimizer='adam')

history = model.fit(train_X, train_Y, batch_size=batch_size,validation_data=(valid_X,valid_Y), epochs = epochs)
            
x = np.array(train[cols2])
x = x.reshape((x.shape[0],x.shape[1],1))
predict = model.predict(x)

train['predict'] = predict.reshape(-1).tolist()

In [ ]:
y_vloss = history.history['val_loss']
y_loss = history.history['loss']

x_len = np.arange(len(y_loss))
plt.plot(x_len, y_vloss, marker='.', c='red', label="Validation-set Loss")
plt.plot(x_len, y_loss, marker='.', c='blue', label="Train-set Loss")

plt.legend(loc='upper right')
plt.grid()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()


In [ ]:
root = '/home/jovyan/WIDE_HOME/model'
### 점수 측정 및 모델 저장 
cnt = 0.0
for p in range(1001) :
    cnt += abs((train['label'][p] - train['predict'][p]) / train['label'][p])
save_pickle(root+'/model/tree/model.pkl', model)
print(cnt)

In [ ]:
#################################

In [ ]:
train2 = train[cols1]

In [ ]:
# Dense 심층 레이어 사용 RNN 모델 : 예측값이 높음
train2_df, valid2_df = train_test_split(train2, test_size=0.2) 
train2_X, train2_Y = np.array(train2_df.iloc[:,:-1]), np.array(train2_df['label']) 
valid2_X, valid2_Y = np.array(valid2_df.iloc[:,:-1]), np.array(valid2_df['label'])

In [ ]:
index = 0
alpha = []
validation_data1 = (valid2_X, valid2_Y)
for i in range(1, 5):
    for j in range(1, 5):
        for k in range(1, 5):
            print(i,j,k)
            model = Sequential()
            model.add(Dense(i,input_dim=2, activation='relu',kernel_initializer='uniform'))
            model.add(Dense(j, activation='relu'))
            model.add(Dense(k, activation='relu'))
            model.add(Dense(1))
            model.compile(optimizer='adam', loss='mse')

            history1 = model.fit(train2_X,train2_Y, validation_data  = validation_data1, epochs=80, batch_size=1000)

            x1 = np.array(train[cols2])
            predict = model.predict(x1)

            train2['predict'] = predict.reshape(-1).tolist()
            ### 모델 저장 
            cnt = 0.0
            for p in range(1001) :
                cnt += abs((train2['label'][p] - train2['predict'][p]) / train2['label'][p])
            if (cnt / 1000) <= 0.7:
                alpha.append('{}, {}, {} : {}'.format(i,j,k,cnt/1000))
                y_vloss = history1.history['val_loss']
                y_loss = history1.history['loss']

                x_len = np.arange(len(y_loss))
                plt.plot(x_len, y_vloss, marker='.', c='red', label="Validation-set Loss")
                plt.plot(x_len, y_loss, marker='.', c='blue', label="Train-set Loss")

                plt.legend(loc='upper right')
                plt.grid()
                plt.xlabel('epoch')
                plt.ylabel('loss')
                plt.show()

                save_pickle(root+'/model/tree/model_Dense{}{}{}.pkl'.format(i,j,k), model)
                index += 1          